In [1]:
import numpy as np 
import pandas as pd
import tmap as tm
from math import log10

from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.AtomPairs import Pairs
from mxfp import mxfp
from map4 import MAP4Calculator
from mhfp.encoder import MHFPEncoder

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from faerun import Faerun

from tqdm import tqdm
tqdm.pandas()

#### Clean dataset (kekulization, remove duplicates)

In [2]:
def SaltRemover(smiles):
    return max(smiles.split('.'), key=len)

In [3]:
df = pd.read_csv('data/erb2.csv', sep=';')
df = df[['Smiles', 'Molecule ChEMBL ID', 'Standard Value', 'Standard Type', 'Standard Relation', 'Standard Units']]
df = df.dropna(subset=['Smiles', 'Standard Value', 'Standard Type', 'Standard Relation', 'Standard Units'])
df = df[df['Standard Units'] == 'nM']
df = df[df['Standard Type'] == 'IC50']
df = df[df['Standard Relation'] == "'='"]
df['Smiles'] = df.Smiles.apply(SaltRemover)
df['ROMol'] = df.Smiles.apply(Chem.MolFromSmiles)
df['Smiles'] = df.ROMol.apply(lambda x: Chem.MolToSmiles(x, kekuleSmiles=True))
df = df.drop_duplicates(subset=['Smiles'])
df

,Smiles,Molecule ChEMBL ID,Standard Value,Standard Type,Standard Relation,Standard Units,ROMol
1,O=C(/C=C/CN1CCSCC1)NC1=CC=C2N=CN=C(NC3=CC=CC(B...,CHEMBL93537,2808.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eef710>
8,COC1=CC=C(S(=O)(=O)N2CCCCC2C(=O)NO)C=C1,CHEMBL140575,670.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eef7b0>
9,O=C(O[C@H]1CN[C@H](C#CC2=CC3=NC=NC(NC4=CC=C5C(...,CHEMBL500217,60.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eefcb0>
12,O=C(/C=C/CN1CCOCC1)N1CCOC2=CC3=NC=NC(NC4=CC=C(...,CHEMBL2441561,996.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eefc10>
13,O=C1CCN(CC2=C3C(NC4=CC=C5C(=C4)C=NN5CC4=CC=CC(...,CHEMBL233324,750.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eef760>
...,...,...,...,...,...,...,...
3931,C[C@@H](NC1=NC=NC2=C1C=C(C1=CC=C(NCC3=CC=C(C(=...,CHEMBL3932057,100.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5f990>
3932,C[C@@H](NC1=NC=NC2=C1C=C(C1=CC=C(CNCC3=CC=C(C(...,CHEMBL3968968,100.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5f9e0>
3935,C=CC(=O)NC1=CC(C2=CNC3=NC=NC(NC4=CC=C(OCC5=CC=...,CHEMBL4757145,15.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5fa80>
3937,C=CC(=O)NC1=CC=CC(C2=CNC3=NC=NC(NC4=CC=C(OCC5=...,CHEMBL4787483,11.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5fad0>


Add molecular properties

In [4]:
df['log(IC50)'] = df['Standard Value'].apply(log10)
df['MW'] = df.ROMol.apply(rdMolDescriptors.CalcExactMolWt)
df['HAC'] = df.ROMol.apply(rdMolDescriptors.CalcNumHeavyAtoms)
df['FCsp3'] = df.ROMol.apply(rdMolDescriptors.CalcFractionCSP3)
df

,Smiles,Molecule ChEMBL ID,Standard Value,Standard Type,Standard Relation,Standard Units,ROMol,log(IC50),MW,HAC,FCsp3
1,O=C(/C=C/CN1CCSCC1)NC1=CC=C2N=CN=C(NC3=CC=CC(B...,CHEMBL93537,2808.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eef710>,3.448397,483.072843,30,0.227273
8,COC1=CC=C(S(=O)(=O)N2CCCCC2C(=O)NO)C=C1,CHEMBL140575,670.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eef7b0>,2.826075,314.093643,21,0.461538
9,O=C(O[C@H]1CN[C@H](C#CC2=CC3=NC=NC(NC4=CC=C5C(...,CHEMBL500217,60.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eefcb0>,1.778151,578.210010,42,0.281250
12,O=C(/C=C/CN1CCOCC1)N1CCOC2=CC3=NC=NC(NC4=CC=C(...,CHEMBL2441561,996.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eefc10>,2.998259,483.147345,34,0.291667
13,O=C1CCN(CC2=C3C(NC4=CC=C5C(=C4)C=NN5CC4=CC=CC(...,CHEMBL233324,750.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eef760>,2.875061,484.213536,36,0.230769
...,...,...,...,...,...,...,...,...,...,...,...
3931,C[C@@H](NC1=NC=NC2=C1C=C(C1=CC=C(NCC3=CC=C(C(=...,CHEMBL3932057,100.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5f990>,2.000000,478.211724,36,0.107143
3932,C[C@@H](NC1=NC=NC2=C1C=C(C1=CC=C(CNCC3=CC=C(C(...,CHEMBL3968968,100.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5f9e0>,2.000000,492.227374,37,0.137931
3935,C=CC(=O)NC1=CC(C2=CNC3=NC=NC(NC4=CC=C(OCC5=CC=...,CHEMBL4757145,15.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5fa80>,1.176091,583.209865,42,0.161290
3937,C=CC(=O)NC1=CC=CC(C2=CNC3=NC=NC(NC4=CC=C(OCC5=...,CHEMBL4787483,11.0,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5fad0>,1.041393,496.141452,36,0.037037


#### Select top 100 closest compounds to Afatinib

Calculate MAP4 fingerprint for library and Afatinib

In [5]:
MAP4 = MAP4Calculator(dimensions=2048)

afatinib_smiles = 'CN(C)C/C=C/C(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(=C(C=C3)F)Cl)O[C@H]4CCOC4'
afatinib_mol = Chem.MolFromSmiles(afatinib_smiles)
afatinib_map4 = MAP4.calculate(afatinib_mol)

df['MAP4'] = df.ROMol.progress_apply(MAP4.calculate)

100%|██████████| 1945/1945 [00:30<00:00, 63.77it/s]


Calculate the distance between every compound and Afatinib

In [6]:
ENC = tm.Minhash(2048)

df['Afatinib_dist'] = df.MAP4.apply(lambda x: ENC.get_distance(x, afatinib_map4))
df = df.sort_values(by=['Afatinib_dist'])
df = df.head(200)
df.reset_index(drop=True, inplace=True)
df

,Smiles,Molecule ChEMBL ID,Standard Value,Standard Type,Standard Relation,Standard Units,ROMol,log(IC50),MW,HAC,FCsp3,MAP4,Afatinib_dist
0,CN(C)C/C=C/C(=O)NC1=CC2=C(NC3=CC=C(F)C(Cl)=C3)...,CHEMBL2347958,14.00,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e4f8f0>,1.146128,485.162996,34,0.291667,"[5547132, 1285846, 6302954, 1288122, 8735147, ...",0.000000
1,CN(C)C/C=C/C(=O)NC1=CC2=C(NC3=CC=C(F)C(Cl)=C3)...,CHEMBL1173655,73.72,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e4e6c0>,1.867585,485.162996,34,0.291667,"[5547132, 1285846, 6302954, 1288122, 8735147, ...",0.000000
2,CN(C)C/C=C(\F)C(=O)NC1=CC2=C(NC3=CC=C(F)C(Cl)=...,CHEMBL3357641,149.40,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5b620>,2.174351,503.153574,35,0.291667,"[5547132, 177484, 6302954, 1288122, 8735147, 2...",0.265137
3,CN(C)C/C=C(/F)C(=O)NC1=CC2=C(NC3=CC=C(F)C(Cl)=...,CHEMBL3357642,97.10,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5d800>,1.987219,503.153574,35,0.291667,"[5547132, 177484, 6302954, 1288122, 8735147, 2...",0.265137
4,COC[C@@H](C)OC1=CC2=NC=NC(NC3=CC=C(F)C(Cl)=C3)...,CHEMBL4577883,148.30,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e5c030>,2.171141,487.178646,34,0.291667,"[5547132, 20494610, 6302954, 1288122, 8735147,...",0.352539
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,CCOC1=CC2=NC=NC(NC3=CC=C4C(=C3)C=NN4CC3=CC=CC=...,CHEMBL3950966,30.00,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e53bc0>,1.477121,577.316523,43,0.294118,"[5183631, 3346815, 4702370, 1738303, 8735147, ...",0.850098
196,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=NC=CN4C)C(C...,CHEMBL175840,625.00,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5eef2b0>,2.795880,559.209866,40,0.241379,"[3258623, 3346815, 1182065, 4382284, 5348810, ...",0.850098
197,CCOC1=CC2=NC=NC(NC3=CC=C4C(=C3)C=NN4CC3=CC=CC=...,CHEMBL3923228,19.00,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e92260>,1.278754,547.269573,41,0.250000,"[5547132, 3346815, 4702370, 1738303, 8735147, ...",0.850586
198,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(N(C)CC4=CC=CC=C4...,CHEMBL175552,2100.00,IC50,'=',nM,<rdkit.Chem.rdchem.Mol object at 0x7f25f5e8e760>,3.322219,568.235352,41,0.218750,"[3258623, 3346815, 1182065, 4382284, 4326848, ...",0.850586


#### TMAP visualization

Define a function that fills a quadratic matrix of arbitrary length with empty strings

In [7]:
def EmptyStringMatrix(length):

    empty_string_matrix = []

    for i in range(length):
        empty_string_list = []
        for j in range(length):
            empty_string_list.append('')
        empty_string_matrix.append(empty_string_list)
    
    return empty_string_matrix

Define a function that generate all possible unique pairs of SMILES displayed as reaction SMILES

In [8]:
smiles_list = df.Smiles.values.tolist()

def PairwiseReactionSMILES(smiles_list):

    pwrs = EmptyStringMatrix(len(smiles_list))

    for i in range(len(smiles_list)):
        for j in range(i, len(smiles_list)):
            pwrs[i][j] = f'{smiles_list[i]}>>{smiles_list[j]}'
            pwrs[j][i] = f'{smiles_list[i]}>>{smiles_list[j]}'
    
    return pwrs

reaction_smiles = pd.DataFrame(PairwiseReactionSMILES(smiles_list)).to_numpy().flatten()

Calculate molecule pair properties

In [9]:
def MeanProperty(list_of_properties):

    pairwise_difference = np.zeros((len(list_of_properties), len(list_of_properties)))

    for i in range(len(list_of_properties)):
        for j in range(i, len(list_of_properties)):
            pairwise_difference[i, j] = (list_of_properties[i] + list_of_properties[j])/2
            pairwise_difference[j, i] = (list_of_properties[i] + list_of_properties[j])/2
    
    return pairwise_difference

In [10]:
def DifferenceProperty(list_of_properties):

    pairwise_difference = np.zeros((len(list_of_properties), len(list_of_properties)))

    for i in range(len(list_of_properties)):
        for j in range(i, len(list_of_properties)):
            pairwise_difference[i, j] = (abs(list_of_properties[i] - list_of_properties[j]))
            pairwise_difference[j, i] = (abs(list_of_properties[i] - list_of_properties[j]))
    
    return pairwise_difference

In [11]:
pairwise_mw = pd.DataFrame(MeanProperty(df.MW.values.tolist())).to_numpy().flatten()
pairwise_hac = pd.DataFrame(MeanProperty(df.HAC.values.tolist())).to_numpy().flatten()
pairwise_fcsp3 = pd.DataFrame(MeanProperty(df.FCsp3.values.tolist())).to_numpy().flatten()
pairwise_activity = pd.DataFrame(DifferenceProperty(df['Standard Value'].values.tolist())).to_numpy().flatten()
pairwise_activity_log50 = pd.DataFrame(DifferenceProperty(df['log(IC50)'].values.tolist())).to_numpy().flatten()

Define pairwise label

In [12]:
def PairwiseLabel(labels):

    label = EmptyStringMatrix(len(labels))

    for i in range(len(labels)):
        for j in range(i, len(labels)):
            label[i][j] = f'{labels[i]} / {labels[j]}'
            label[j][i] = f'{labels[i]} / {labels[j]}'
    
    return label

In [13]:
pairwise_label = pd.DataFrame(PairwiseLabel(df['Molecule ChEMBL ID'].values.tolist())).to_numpy().flatten()

Define function for merged MinHashed fingerprints

In [14]:
def MergeMAP4(map4_list):

    map4_matrix = []
    for i in range(len(map4_list)):
        map4_row = []
        for j in range(len(map4_list)):
            map4_row.append(np.minimum(map4_list[i], map4_list[j]))
        map4_matrix.append(map4_row)
    return map4_matrix

Calculate merged MAP4 for all molecular pairs

In [15]:
pairwise_map4 = pd.DataFrame(MergeMAP4(df.MAP4.values.tolist())).to_numpy().flatten()

Prepare final dataframe for TMAP

In [16]:
df_tmap = pd.DataFrame(list(zip(pairwise_label, reaction_smiles, pairwise_mw, pairwise_hac, pairwise_fcsp3, pairwise_activity, pairwise_activity_log50, pairwise_map4)), 
                                columns=['Label', 'ReactionSMILES', 'uMW', 'uHAC', 'uFCsp3', 'dIC50', 'dlog(IC50)', 'MAP4'])

TMAP

Layout

In [17]:
lf = tm.LSHForest(1024, 64)

merged_map4 = np.array(df_tmap['MAP4'])
fps = []

for i in merged_map4:
    vec = tm.VectorUint(i)
    fps.append(vec)

lf.batch_add(fps)
lf.index()

cfg = tm.LayoutConfiguration() #configuration parameters for tmap layout
cfg.node_size = 1 / 30 #size of nodes which affects the magnitude of their repelling force. Decreasing this values generally resolves overlaps in a very crowded tree
cfg.mmm_repeats = 2 #number of repeats of the per-level layout algorithm
cfg.sl_extra_scaling_steps = 5 #sets the number of repeats of the scaling
cfg.k = 45 #number of nearest neighbours used to create the k-nearest neighbour graph
cfg.sl_scaling_type = tm.RelativeToAvgLength #Defines the relative scale of the graph
x, y, s, t, _ = tm.layout_from_lsh_forest(lf, cfg)

Colormaps and customized labels

In [18]:
labels = []

for i, row in df_tmap.iterrows():
    labels.append(
            row["ReactionSMILES"]
            + "__"
            + f'{row["ReactionSMILES"]}'
            + "__"
            + f'Label: {row["Label"]}'
            + "__"
            + f'dlog(IC50): {row["dlog(IC50)"]}'
        )

Labels of sets

In [23]:
f = Faerun(
    view="front", 
    coords=False,
    title="",
    clear_color='#FFFFFF',
)

f.add_scatter(
    "MAP4_TMAP",
    {
        "x": x,
        "y": y,
        "c": [
            df_tmap.uMW.values.tolist(),
            df_tmap.uHAC.values.tolist(), 
            df_tmap.uFCsp3.values.tolist(),
            df_tmap.dIC50.values.tolist(),
            df_tmap['dlog(IC50)'].values.tolist()
            ],
        "labels": labels,
    },
    shader="sphere",
    point_scale=5,
    max_point_size=20,
    legend_labels=[None, None, None, None, None],
    categorical=[False, False, False, False, False],
    colormap=['rainbow_r', 'rainbow_r', 'rainbow_r', 'rainbow_r', 'rainbow_r'],
    series_title=['uMW', 'uHAC', 'uFCsp3', 'dIC50', 'dlog(IC50)'],
    has_legend=True,
)
f.add_tree("MAP4_TMAP_tree", {"from": s, "to": t}, point_helper="MAP4_TMAP")
f.plot('plots/Erb2_MAP4_TMAP', template='reaction_smiles')

/home/markus/Developer/Code/mxfp_python/benchmark/plots/Erb2_MAP4_TMAP.html

#### TMAP of molecules

In [ ]:
lf = tm.LSHForest(1024, 64)

merged_map4 = np.array(df['MAP4'])
fps = []

for i in merged_map4:
    vec = tm.VectorUint(i)
    fps.append(vec)

lf.batch_add(fps)
lf.index()

cfg = tm.LayoutConfiguration() #configuration parameters for tmap layout
cfg.node_size = 1 / 15 #size of nodes which affects the magnitude of their repelling force. Decreasing this values generally resolves overlaps in a very crowded tree
cfg.mmm_repeats = 2 #number of repeats of the per-level layout algorithm
cfg.sl_extra_scaling_steps = 5 #sets the number of repeats of the scaling
cfg.k = 45 #number of nearest neighbours used to create the k-nearest neighbour graph
cfg.sl_scaling_type = tm.RelativeToAvgLength #Defines the relative scale of the graph
x, y, s, t, _ = tm.layout_from_lsh_forest(lf, cfg)

In [ ]:
f = Faerun(
    view="front", 
    coords=False,
    title="",
)

f.add_scatter(
    "MAP4_TMAP",
    {
        "x": x,
        "y": y,
        "c": [
            df.MW.values.tolist(),
            df.HAC.values.tolist(), 
            df.FCsp3.values.tolist(),
            df['Standard Value'].values.tolist()
            ],
        "labels": df.Smiles.values.tolist(),
    },
    shader="smoothCircle",
    point_scale=4,
    max_point_size=20,
    legend_labels=[None, None, None, None],
    categorical=[False, False, False, False],
    colormap=['rainbow_r', 'rainbow_r', 'rainbow_r', 'bwr'],
    series_title=['μMW', 'μHAC', 'μFCsp3', 'Activity'],
    has_legend=True,
)
f.add_tree("MAP4_TMAP_tree", {"from": s, "to": t}, point_helper="MAP4_TMAP")
f.plot('plots/Erb2_mols_MAP4_TMAP', template='smiles')